In [6]:
import tensorflow as tf
import os
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.18.0


In [7]:
gpus=tf.config.experimental.list_physical_devices('GPU')
if gpus:
  print("no of gpu available:",len(gpus))
  for gpu in gpus:
    print("GPU:",gpu)
else:
  print("No gpu available using cpu.")


no of gpu available: 1
GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [8]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [9]:
 # Basic text cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    return text

In [10]:
 df = pd.read_csv("spam.csv", encoding="latin-1")[["v1", "v2"]]
df.columns = ["label", "message"]
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [12]:
 df['message'] = df['message'].apply(clean_text)
from sklearn.feature_extraction.text import TfidfVectorizer
# Convert text to vector using Bag-of-Words
vectorizer = TfidfVectorizer(max_features=300)
X = vectorizer.fit_transform(df['message']).toarray()
y = df['label'].values

In [13]:
 X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
 model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │         4,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,961 (19.38 KB)

 Trainable params: 4,961 (19.38 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
 history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7926 - loss: 0.5851 - val_accuracy: 0.8565 - val_loss: 0.3173
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8730 - loss: 0.2476 - val_accuracy: 0.9283 - val_loss: 0.1965
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9550 - loss: 0.1556 - val_accuracy: 0.9619 - val_loss: 0.1414
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9751 - loss: 0.0975 - val_accuracy: 0.9619 - val_loss: 0.1143
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9785 - loss: 0.0729 - val_accuracy: 0.9596 - val_loss: 0.1120
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9822 - loss: 0.0638 - val_accuracy: 0.9619 - val_loss: 0.1084
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9851 - loss: 0.0546 - val_accuracy: 0.9596 - val_loss: 0.1063
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9827 - loss: 0.0471 - val_accuracy: 0

In [16]:
 loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {accuracy:.4f}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9712 - loss: 0.0898
✅ Test Accuracy: 0.9704


In [17]:
def predict_message_tf(msg):
    msg_clean = clean_text(msg)
    msg_vector = vectorizer.transform([msg_clean]).toarray()
    prediction = model.predict(msg_vector)[0][0]
    return "📩 Spam" if prediction > 0.5 else "✅ Not Spam"

print(predict_message_tf("Get laptop free on indian website shop now"))
print(predict_message_tf("Meeting at 5 pm, bring your notes."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
📩 Spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
✅ Not Spam


In [18]:
model.save('model.h5')

In [19]:
from tensorflow.keras.models import load_model

In [24]:
# Load the saved model
loaded_model = load_model("model.h5")

def predict_with_loaded_model(msg):
    msg_clean = clean_text(msg)
    msg_vector = vectorizer.transform([msg_clean]).toarray()
    prediction = loaded_model.predict(msg_vector)[0][0]
    return "📩 Spam" if prediction > 0.5 else "✅ Not Spam"

# Example usage:
print(predict_with_loaded_model("Get laptop free on indian website shop now."))
print(predict_with_loaded_model("your mom is calling you."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
📩 Spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
✅ Not Spam
